In [ ]:
from pathlib import Path
from tqdm.notebook import tqdm
import csv, json, cv2, numpy as np

IMG_DIR   = Path("")
LBL_DIR   = Path("")
CSV_PATH  = Path("")
OUT_DIR   = Path("")
NOISE_STD = 0

def read_scene_polygons(csv_path: Path) -> dict[str, np.ndarray]:
    polys = {}
    with csv_path.open(newline="", encoding="utf-8") as fh:
        for row in csv.DictReader(fh):
            scene = row["filename"].split("_")[1]
            if scene in polys:
                continue
            verts = json.loads(row["region_shape_attributes"])
            pts   = np.column_stack((verts["all_points_x"],
                                     verts["all_points_y"])).astype(np.int32)
            polys[scene] = pts
    if not polys:
        raise ValueError("no mask polygons")
    return polys


def load_yolo_boxes(lbl_path: Path, W: int, H: int) -> list[tuple[int, int, int, int]]:
    boxes = []
    with lbl_path.open() as f:
        for ln in f:
            p = ln.split()
            if len(p) < 5:
                continue
            _, xc, yc, w, h = map(float, p[:5])
            xc, yc, w, h = xc*W, yc*H, w*W, h*H
            x1, y1 = int(max(xc - w/2, 0)), int(max(yc - h/2, 0))
            x2, y2 = int(min(xc + w/2, W-1)), int(min(yc + h/2, H-1))
            boxes.append((x1, y1, x2, y2))
    return boxes


def apply_mask(img: np.ndarray,
               roi_poly: np.ndarray,
               boxes: list[tuple[int, int, int, int]],
               noise_std: float) -> np.ndarray:
    H, W = img.shape[:2]
    med  = np.median(img.reshape(-1, 3), axis=0).astype(np.uint8)

    roi = np.zeros((H, W), np.uint8)
    cv2.fillPoly(roi, [roi_poly], 255)
    ignore = cv2.bitwise_not(roi)

    out = img.copy()
    out[ignore == 255] = med

    if noise_std > 0:
        noise = np.random.normal(0, noise_std, img.shape).astype(np.int16)
        tmp   = np.clip(out.astype(np.int16) + noise, 0, 255).astype(np.uint8)
        out[ignore == 255] = tmp[ignore == 255]

    for x1, y1, x2, y2 in boxes:
        if cv2.countNonZero(ignore[y1:y2+1, x1:x2+1]) > 0:
            out[y1:y2+1, x1:x2+1] = img[y1:y2+1, x1:x2+1]

    return out

def mask_dataset(img_dir: Path     = IMG_DIR,
                 lbl_dir: Path|None = LBL_DIR,
                 csv_path: Path    = CSV_PATH,
                 out_dir: Path     = OUT_DIR,
                 noise_std: float  = NOISE_STD) -> None:
    out_dir.mkdir(parents=True, exist_ok=True)
    polys  = read_scene_polygons(csv_path)
    frames = sorted(img_dir.glob("*.jpg"))

    for frame in tqdm(frames, desc="Masking"):
        scene_id = frame.stem.split("_")[1]
        poly     = polys.get(scene_id)
        if poly is None:
            continue

        img = cv2.imread(str(frame))
        if img is None:
            tqdm.write(f"[WARN] unreadable: {frame}")
            continue

        boxes = []
        if lbl_dir is not None:
            lbl_file = lbl_dir / f"{frame.stem}.txt"
            if lbl_file.exists():
                boxes = load_yolo_boxes(lbl_file, *img.shape[1::-1])

        masked = apply_mask(img, poly, boxes, noise_std)
        cv2.imwrite(str(out_dir / frame.name), masked)

In [ ]:
mask_dataset()